In [ ]:
import numpy as np
from scipy.stats import norm
import scipy.linalg as la
import pymc3 as pm

def GetData():
    input = np.genfromtxt("input.csv",delimiter=',')
    output = np.genfromtxt("output.csv",delimiter=',')
    return (input,output)

def Model(x):
    k1 = x[0]
    k2 = 1500.0
    k3 = 1500.0
    k4 = x[1]
    
    m = 100
    
    # masses
    m1 = m
    m2 = 2*m
    m3 = 3*m
    
    M = np.array([[m1,0,0],[0,m2,0],[0,0,m3]])
    K = np.array([[k1+k2,-k2,0],[-k2,k2+k3,-k3],[0,-k3,k3+k4]])
    # some algorihtm to find the eigenvalues from (K-lambda*M) = 0 (I forgot the source sorry)
    a = -m1*m2*m3
    b = K[0,0]*m2*m3+m1*K[1,1]*m3 + m1*m2*K[2,2]
    c = -K[0,0]*K[1,1]*m3 - K[0,0]*K[2,2]*m2 - K[1,1]*K[2,2]*m1+K[1,2]*K[2,1]*m1+K[0,1]*K[1,0]*m3
    d = K[0,0]*K[1,1]*K[2,2]-K[0,0]*K[1,2]*K[2,1]-K[0,1]*K[1,0]*K[2,2]
    
    p = 3*a*c-b**2
    q = 2*b**3-9*a*b*c+27*a**2*d
    phi = np.arccos(-q/(2*np.sqrt(-p**3)))
    y = np.sqrt(-p)*2*np.cos(phi/3+np.array([0,1,2])*2*np.pi/(3))

    return (y-b)/(3*a) # 1x3 array with the natural frequencies

def NatFreqExample_LogLikelihood_SN(theta,data,sigma): # likelihood for standard-normal distributed theta
    theta_phys = 500 + norm.cdf(theta)*3000 # theta ~ U(500,3500)
    return np.log(1/(np.sqrt(2*np.pi*sigma**2))) * np.sum(-(Model(theta_phys)/data)**2/(2*sigma**2))

def NatFreqExample_LogLikelihood_Phys(theta,data,sigma): # likelihood for theta in physical space
    return np.log(1/(np.sqrt(2*np.pi*sigma**2))) * np.sum(-(Model(theta)/data)**2/(2*sigma**2))

: 

In [2]:
input, output = GetData()

with pm.Model() as model:
    # sigma = 1e-1
    k1 = pm.Normal("k1", mu=0, sigma=1)
    k2 = pm.Normal("k2", mu=0, sigma=1)
    obs = pm.MvNormal("obs", mu = np.array([k1,k2]), cov = sigma * np.eye(2), observed = output)

print(model.basic_RVs)
print(model.free_RVs)
print(model.observed_RVs)


You can find the C code in this temporary file: /tmp/theano_compilation_error_qtgln1r_


Exception: ("Compilation failed (return status=1): /usr/bin/ld: /home/jan/.pyenv/versions/3.8.10/lib/libpython3.8.a(obmalloc.o): relocation R_X86_64_PC32 against symbol `stderr@@GLIBC_2.2.5' can not be used when making a shared object; recompile with -fPIC. /usr/bin/ld: final link failed: bad value. collect2: error: ld returned 1 exit status. ", 'FunctionGraph(Elemwise{true_div,no_inplace}(TensorConstant{1.0}, TensorConstant{1.0}))')